In [4]:
import pandas as pd

In [ ]:
GLOBAL_RELIGION_DATA = '../data/world-religions/global_raw.csv'
NATIONAL_RELIGION_DATA = '../data/world-religions/national_raw.csv'
LOCAL_RELIGION_DATA = '../data/world-religions/regional_raw.csv'

In [5]:
TERRORIST_ATTACKS_DATA = '../data/terrorist-attacks/cleaned.csv'

In [9]:
terrorist_attack_tbl = pd.read_csv(TERRORIST_ATTACKS_DATA)
terrorist_attack_tbl = terrorist_attack_tbl.drop(columns='Unnamed: 0')

In [16]:
tbls = list(terrorist_attack_tbl.groupby(('country', 'year')))

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  if __name__ == '__main__':


In [24]:
tbls[0][1]

,date,country,city,killed,injured,description,year
150,2002-04-08,Afghanistan,Jalalabad,9,92,A terrorist bomb explodes near a government co...,2002
241,2002-06-06,Afghanistan,Kabul,2,0,The country's Deputy President and his driver ...,2002
249,2002-06-10,Afghanistan,Kabul,1,0,A government official is stabbed to death by a...,2002
374,2002-09-01,Afghanistan,Kabul,1,3,"Taliban bombing kills one, injures three others.",2002
378,2002-09-05,Afghanistan,Kabul,22,65,Bomb explodes in busy Kabul marketplace. At le...,2002
380,2002-09-06,Afghanistan,Akora Khattak,1,2,Muslim militants disguise a bomb as a toy. On...,2002


In [ ]:
terrorist_attack_tbl.set_index(['year', 'country'])

In [ ]:
religion_data['year'] = religion_data.year.astype('int')

In [ ]:
terrorist_attack_tbl['year'] = terrorist_attack_tbl['year'].astype('int')

In [ ]:
terrorist_attack_tbl.head()

In [ ]:
religion_data

In [ ]:
religion_data

In [ ]:
terrorist_attack_tbl = terrorist_attack_tbl.drop(columns='Unnamed: 0')

In [ ]:
terrorist_attack_tbl.columns = ['date', 'country', 'city', 'killed', 'injured', 'description']

In [ ]:
terrorist_attack_tbl['year'] = terrorist_attack_tbl.date.map(lambda d: d.split('-')[0])

In [ ]:
terrorist_attack_tbl.to_csv('../data/terrorist-attacks/cleaned.csv')